# Querying `DataFrame` 

## Retrieving our Books

In [2]:
import org.apache.spark.sql.types._
val bookSchema = new StructType(Array(
   new StructField("bookID", IntegerType, false),
   new StructField("title", StringType, false),
   new StructField("authors", StringType, false),
   new StructField("average_rating", FloatType, false),
   new StructField("isbn", StringType, false),
   new StructField("isbn13", StringType, false),
   new StructField("language_code", StringType, false),
   new StructField("num_pages", IntegerType, false),
   new StructField("ratings_count", IntegerType, false),
   new StructField("text_reviews_count", IntegerType, false)))

val booksDF = spark.read.format("csv")
                         .schema(bookSchema)
                         .option("inferSchema", "true")
                         .option("header", "true")
                         .load("../data/books.csv")
booksDF.printSchema()

root
 |-- bookID: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- authors: string (nullable = true)
 |-- average_rating: float (nullable = true)
 |-- isbn: string (nullable = true)
 |-- isbn13: string (nullable = true)
 |-- language_code: string (nullable = true)
 |-- num_pages: integer (nullable = true)
 |-- ratings_count: integer (nullable = true)
 |-- text_reviews_count: integer (nullable = true)



import org.apache.spark.sql.types._
bookSchema: org.apache.spark.sql.types.StructType = StructType(StructField(bookID,IntegerType,false), StructField(title,StringType,false), StructField(authors,StringType,false), StructField(average_rating,FloatType,false), StructField(isbn,StringType,false), StructField(isbn13,StringType,false), StructField(language_code,StringType,false), StructField(num_pages,IntegerType,false), StructField(ratings_count,IntegerType,false), StructField(text_reviews_count,IntegerType,false))
booksDF: org.apache.spark.sql.DataFrame = [bookID: int, title: string ... 8 more fields]


## Filtering Rows

### `where`

* Can be applied to filter rows from the `DataFrame`/`DataSet`
* Can take as an argument
  * A `Column` with some criteria
  * A `String` that represents a query
* Returns a new `DataFrame` with the query results

Here we will procure a `Column` either using `df("..")`, `$".."`, `'..`, `col(..)`, or `column(..)` where `..` is the name of the column. Also please open the [column API reference](https://spark.apache.org/docs/2.3.0/api/scala/index.html#org.apache.spark.sql.Column) to see what kind of calls can be made.

In [4]:
val stephenKing = booksDF.where($"authors".contains("Stephen King"))
stephenKing.show(10)

+------+--------------------+--------------------+--------------+----------+-------------+-------------+---------+-------------+------------------+
|bookID|               title|             authors|average_rating|      isbn|       isbn13|language_code|num_pages|ratings_count|text_reviews_count|
+------+--------------------+--------------------+--------------+----------+-------------+-------------+---------+-------------+------------------+
|  4978|Wolves of the Cal...|Stephen King-Bern...|          4.19|141651693X|9781416516934|          eng|      931|       120906|              2640|
|  5094|The Drawing of th...|        Stephen King|          4.23|0451210859|9780451210852|          eng|      463|       163647|              4846|
|  5095|The Waste Lands (...|        Stephen King|          4.24|034082977X|9780340829776|          eng|      584|         1073|                82|
|  5096|Wizard and Glass ...|Stephen King-Dave...|          4.25|0340829788|9780340829783|          eng|      84

stephenKing: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [bookID: int, title: string ... 8 more fields]


### `where` with `like`

Here we will run the same query as above but using `like`, and using the `col` method to procure a column

In [8]:
val stephenKing = booksDF.where(col("authors").like("Stephen King"))
stephenKing.show(10)

+------+--------------------+------------+--------------+----------+-------------+-------------+---------+-------------+------------------+
|bookID|               title|     authors|average_rating|      isbn|       isbn13|language_code|num_pages|ratings_count|text_reviews_count|
+------+--------------------+------------+--------------+----------+-------------+-------------+---------+-------------+------------------+
|  5094|The Drawing of th...|Stephen King|          4.23|0451210859|9780451210852|          eng|      463|       163647|              4846|
|  5095|The Waste Lands (...|Stephen King|          4.24|034082977X|9780340829776|          eng|      584|         1073|                82|
|  5098|The Gunslinger (T...|Stephen King|          3.96|0340829753|9780340829752|          eng|      238|         1598|               189|
|  5399|           The Stand|Stephen King|          4.34|1568495714|9781568495712|          eng|     1344|          412|                33|
|  5415|        'Sal

stephenKing: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [bookID: int, title: string ... 8 more fields]


### Using `where` with a `String`

Here we will use a `String` with a SQL like format, which will be a nice segue to SparkSQL. For more on the SQL calls that can be made with `where`, [here is a handy reference](https://spark.apache.org/docs/2.3.0/api/sql/index.html)

In [12]:
val stephenKing = booksDF.where("title like '%Stephen King%'")
stephenKing.show()

+------+--------------------+--------------------+--------------+----------+-------------+-------------+---------+-------------+------------------+
|bookID|               title|             authors|average_rating|      isbn|       isbn13|language_code|num_pages|ratings_count|text_reviews_count|
+------+--------------------+--------------------+--------------+----------+-------------+-------------+---------+-------------+------------------+
| 10586|The Stephen King ...|Stephen King-John...|          3.99|0739317369|9780739317365|          eng|       11|           55|                 9|
| 10594|Stephen King: Ame...|        George Beahm|          3.79|0836254279|9780836254273|        en-US|      304|           49|                 3|
| 10597|The Illustrated S...|Brian James Freem...|          4.23|1587671166|9781587671166|          eng|      404|           22|                 5|
| 10604|The Body Snatcher...|Jack Finney-Steph...|          3.89|1582881804|9781582881805|          eng|      22

stephenKing: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [bookID: int, title: string ... 8 more fields]


### `filter` 

* `filter` takes a functional approach. 
* `DataFrame` and `Dataset` are the same
  * A `DataFrame` is a `Dataset[Row]`
* Has the same signature as `where` but also has the ability to select information using a Scala function

In [17]:
val stephenKing = booksDF.filter(booksDF("authors").contains("Stephen King"))
stephenKing.show(5)

+------+--------------------+--------------------+--------------+----------+-------------+-------------+---------+-------------+------------------+
|bookID|               title|             authors|average_rating|      isbn|       isbn13|language_code|num_pages|ratings_count|text_reviews_count|
+------+--------------------+--------------------+--------------+----------+-------------+-------------+---------+-------------+------------------+
|  4978|Wolves of the Cal...|Stephen King-Bern...|          4.19|141651693X|9781416516934|          eng|      931|       120906|              2640|
|  5094|The Drawing of th...|        Stephen King|          4.23|0451210859|9780451210852|          eng|      463|       163647|              4846|
|  5095|The Waste Lands (...|        Stephen King|          4.24|034082977X|9780340829776|          eng|      584|         1073|                82|
|  5096|Wizard and Glass ...|Stephen King-Dave...|          4.25|0340829788|9780340829783|          eng|      84

stephenKing: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [bookID: int, title: string ... 8 more fields]


In [26]:
val jkRowling = booksDF.filter("authors like '%J.K. Rowling%'")
jkRowling.show(5)

+------+--------------------+--------------------+--------------+----------+-------------+-------------+---------+-------------+------------------+
|bookID|               title|             authors|average_rating|      isbn|       isbn13|language_code|num_pages|ratings_count|text_reviews_count|
+------+--------------------+--------------------+--------------+----------+-------------+-------------+---------+-------------+------------------+
|     1|Harry Potter and ...|J.K. Rowling-Mary...|          4.56|0439785960|9780439785969|          eng|      652|      1944099|             26249|
|     2|Harry Potter and ...|J.K. Rowling-Mary...|          4.49|0439358078|9780439358071|          eng|      870|      1996446|             27613|
|     3|Harry Potter and ...|J.K. Rowling-Mary...|          4.47|0439554934|9780439554930|          eng|      320|      5629932|             70390|
|     4|Harry Potter and ...|        J.K. Rowling|          4.41|0439554896|9780439554893|          eng|      35

jkRowling: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [bookID: int, title: string ... 8 more fields]


## `filter` using functional programming

* We can use functional programming to filter out a `DataFrame`/`Dataset`
* That includes a `filter` that will query information based on `Row`
* Seek out the [Row Spark API](https://spark.apache.org/docs/2.3.0/api/scala/index.html#org.apache.spark.sql.Row) on how to dig into the row and get the information you desire
* Here we will use something different called `getAs[T]` where `[T]` is the generic type

In [31]:
val badBooks = booksDF.filter(row => row.getAs[Float]("average_rating") < 3)
badBooks.show(5)

+------+--------------------+--------------------+--------------+----------+-------------+-------------+---------+-------------+------------------+
|bookID|               title|             authors|average_rating|      isbn|       isbn13|language_code|num_pages|ratings_count|text_reviews_count|
+------+--------------------+--------------------+--------------+----------+-------------+-------------+---------+-------------+------------------+
|   159|Dinner with Anna ...|    Gloria Goldreich|          2.96|0778322270|9780778322276|          eng|      368|          400|                64|
|   799|Out to Eat London...|Lonely Planet-Mar...|           0.0|1740592050|9781740592055|          eng|      295|            0|                 0|
|  1302|Juiced Official S...|          Doug Walsh|           0.0|0744005612|9780744005615|          eng|      112|            0|                 0|
|  1584|Cliffs Notes on A...|    W. John Campbell|          2.33|0822007762|0049086007763|          eng|       8

badBooks: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [bookID: int, title: string ... 8 more fields]


Query columns using expr

Query columns using selectExpr

Sorting


GroupBy and Aggregate functions

In [19]:
val subset = booksDF.select($"ratings_count", col("title"), 'authors)
subset.show()

+-------------+--------------------+--------------------+
|ratings_count|               title|             authors|
+-------------+--------------------+--------------------+
|      1944099|Harry Potter and ...|J.K. Rowling-Mary...|
|      1996446|Harry Potter and ...|J.K. Rowling-Mary...|
|      5629932|Harry Potter and ...|J.K. Rowling-Mary...|
|         6267|Harry Potter and ...|        J.K. Rowling|
|      2149872|Harry Potter and ...|J.K. Rowling-Mary...|
|        38872|Harry Potter Boxe...|J.K. Rowling-Mary...|
|           18|Unauthorized Harr...|W. Frederick Zimm...|
|        27410|Harry Potter Coll...|        J.K. Rowling|
|         3602|The Ultimate Hitc...|       Douglas Adams|
|       240189|The Ultimate Hitc...|       Douglas Adams|
|         4416|The Hitchhiker's ...|       Douglas Adams|
|         1222|The Hitchhiker's ...|Douglas Adams-Ste...|
|         2801|The Ultimate Hitc...|       Douglas Adams|
|       228522|A Short History o...|Bill Bryson-Willi...|
|         6993

subset: org.apache.spark.sql.DataFrame = [ratings_count: int, title: string ... 1 more field]


In [20]:
val subset = booksDF.select($"ratings_count".as("num_ratings"), col("title"), 'authors)
subset.show()

+-----------+--------------------+--------------------+
|num_ratings|               title|             authors|
+-----------+--------------------+--------------------+
|    1944099|Harry Potter and ...|J.K. Rowling-Mary...|
|    1996446|Harry Potter and ...|J.K. Rowling-Mary...|
|    5629932|Harry Potter and ...|J.K. Rowling-Mary...|
|       6267|Harry Potter and ...|        J.K. Rowling|
|    2149872|Harry Potter and ...|J.K. Rowling-Mary...|
|      38872|Harry Potter Boxe...|J.K. Rowling-Mary...|
|         18|Unauthorized Harr...|W. Frederick Zimm...|
|      27410|Harry Potter Coll...|        J.K. Rowling|
|       3602|The Ultimate Hitc...|       Douglas Adams|
|     240189|The Ultimate Hitc...|       Douglas Adams|
|       4416|The Hitchhiker's ...|       Douglas Adams|
|       1222|The Hitchhiker's ...|Douglas Adams-Ste...|
|       2801|The Ultimate Hitc...|       Douglas Adams|
|     228522|A Short History o...|Bill Bryson-Willi...|
|       6993|Bill Bryson's Afr...|         Bill 

subset: org.apache.spark.sql.DataFrame = [num_ratings: int, title: string ... 1 more field]


In [21]:
subset.printSchema()

root
 |-- num_ratings: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- authors: string (nullable = true)



## Sorting the data

We can sort the data with the `sort` method providing a column. The default is ascending order

In [31]:
harryPotters.sort($"# num_pages").show()

+--------------------+----------+-----------+
|               title|      isbn|# num_pages|
+--------------------+----------+-----------+
|Harry Potter und ...|3895849618|         13|
|Unauthorized Harr...|0976540606|        152|
|Harry Potter Boxe...|0439434866|       1820|
|Mapping the World...|1932100598|        195|
|Mugglenet.Com's W...|1569755833|        216|
|Looking for God i...|1414306342|        234|
|Harry Potter Scho...|043932162X|        240|
|Harry Potter and ...|0812694554|        243|
|Harry Potter and ...|158234681X|        250|
|Harry Potter Y La...|0613359607|        254|
|Harry Potter Boxe...|0439682584|       2690|
|Harry Potter y la...|8498380138|        288|
|Harry Potter and ...|0439554934|        320|
|Harry Potter und ...|3551354014|        334|
|Harry Potter Coll...|0439827604|       3342|
|Harry Potter and ...|0439064864|        341|
|Harry Potter et l...|2070541304|        349|
|Harry Potter und ...|3551552096|        351|
|Harry Potter and ...|0439554896| 

## Bring in some more functions

* For descending order we need some help, a common import is the `import org.apache.spark.sql.functions._` package
* Contains a wide range of functions that compliment what is in `DataFrame` API
* Nearly all of the functions of this Scala `object` is 

In [32]:
import org.apache.spark.sql.functions._

import org.apache.spark.sql.functions._


## Sorting in Descending

* For descending order we need some help, a common import is the `import org.apache.spark.sql.functions._` package
* Contains a wide range of functions that compliment what is in `DataFrame` API

In [37]:
harryPotters.sort(desc("# num_pages")).show()

+--------------------+----------+-----------+
|               title|      isbn|# num_pages|
+--------------------+----------+-----------+
|J.K. Rowling's Ha...|0826452329|         96|
|Harry Potter y la...|8478887423|        896|
|Harry Potter y la...|8478888845|        893|
|Harry Potter and ...|0439358078|        870|
|Harry Potter and ...|0747584664|        768|
|Harry Potter and ...|0439785960|        652|
|Harry Potter and ...|074754624X|        636|
|Harry Potter e il...|888451049X|        627|
|Harry Potter y el...|8478889930|        602|
|Harry Potter and ...|074757362X|        480|
|Harry Potter und ...|355155210X|        448|
|Harry Potter and ...|043965548X|        435|
|Harry Potter and ...|0786222727|        424|
|Ultimate Unoffici...|0972393617|        412|
|Harry Potter ve S...|3570211029|        403|
|Гарри Поттер и фи...|535300308X|        400|
|The Science Of Ha...|0755311515|        374|
|Harry Potter y el...|8478886559|        359|
|Harry Potter ve F...|3570211010| 

## Inappropriate Types

The `# num_pages` column is not in a numerical format we can prove it by calling `printSchema`

In [38]:
harryPotters.printSchema()

root
 |-- title: string (nullable = true)
 |-- isbn: string (nullable = true)
 |-- # num_pages: string (nullable = true)



In [ ]:
df = harryPotters.withColumn("num_pages", df("num_pages").cast(IntegerType)).drop("# num_pages")

In [31]:
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types.IntegerType
val converted = harryPotters.withColumn("# num_pages", $"# num_pages".cast(IntegerType))
converted.printSchema()

root
 |-- title: string (nullable = true)
 |-- isbn: string (nullable = true)
 |-- # num_pages: integer (nullable = true)



import org.apache.spark.sql.functions._
import org.apache.spark.sql.types.IntegerType
converted: org.apache.spark.sql.DataFrame = [title: string, isbn: string ... 1 more field]


val total = converted.agg(sum($"# num_pages"))
total.show()

In [40]:
harryPotters.select("title").where($"title".contains("Prisoner")).show(20, false)

+-----------------------------------------------------------+
|title                                                      |
+-----------------------------------------------------------+
|Harry Potter and the Prisoner of Azkaban (Harry Potter  #3)|
|Harry Potter and the Prisoner of Azkaban (Harry Potter  #3)|
+-----------------------------------------------------------+



In [ ]:
collect_list($"title")

In [54]:
booksDF.groupBy($"authors").agg(collect_list($"title").alias("titles")).show()

+--------------------+--------------------+
|             authors|              titles|
+--------------------+--------------------+
|Abraham Lincoln-D...|[Speeches and Wri...|
|    Amanda Eyre Ward|    [How to Be Lost]|
|         Ann Beattie|[The Doctor's Hou...|
|         Ann Rinaldi|[A Break with Cha...|
|Charles Dickens-S...|[A Tale of Two Ci...|
|          Dava Sobel|[Galileo's Daught...|
|        Doug Stanton|[In Harm's Way: T...|
|     Eric Klinenberg|[Heat Wave: A Soc...|
|Gayle Lynds-Rober...|[The Altman Code ...|
|Haruki Murakami-U...|    [Naokos Lächeln]|
|          Ian Ogilvy|[Measle and the D...|
|J.E. Austen Leigh...|[A Memoir of Jane...|
|        Jack Meadows|[The Future of th...|
|          James Frey|[A Million Little...|
|Johanna Hurwitz-V...|[Anne Frank: Life...|
|John  Baxter-Mel Bay|[Deluxe Encyclope...|
|Jonathan Swift-YKids|[Gulliver's Travels]|
|     Karen Armstrong|[A History of God...|
|Laura  Jordan-San...|   [Anhelos ocultos]|
|    Laurence Olivier|[Confessio

Lab: What are the harry potter's books average rating?